<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/Opt_T7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Definimos las funciones para calcular el gradiente y Hessiano de la función de Rosembrok.

In [203]:
def Rosembrok(X):#X es un vector.
    return np.sum(100*(X[1:]-X[:-1]**2)**2+(1-X[:-1])**2)
def Rosembrok_grad(X):
    n=len(X)
    grad=np.zeros(n)
    grad[1:-1]=-400*X[1:-1]*(X[2:]-X[1:-1]**2)+200*(X[1:-1]-X[:-2]**2)-2*(1-X[1:-1])
    grad[0]=-400*X[0]*(X[1]-X[0]**2)-2*(1-X[0])
    grad[-1]=200*(X[-1]-X[-2]**2)
    return grad
def Rosembrok_Hessian(X):
    banda_s=X[:-1]
    H=np.diag(banda_s,1)
    H=-400*H+-400*H.T
    main_diag=-400*(X[2:]-X[1:-1]**2)+800*(X[1:-1]**2)+202
    np.fill_diagonal(H[1:-1,1:-1],main_diag)
    H[0,0]=(-400*(X[1]-X[0]**2)+800*(X[0]**2)+2)
    H[-1,-1]=200
    H[0,1]=-400*(X[1])
    H[-1,-2]=-400*X[-2]
    return(H)

Definimos las funciones para calcular el gradiente y Hessiano de la función de Wood.


In [104]:
def Wood(X):
    return (100*(X[0]**2-X[1])**2+(X[0]-1)**2+(X[2]-1)**2+90*(X[2]**2-X[3])**2+10.1*((X[1]-1)**2+(X[3]-1)**2)+19.8*(X[1]-1)*(X[3]-1))
def Wood_grad(X):
    G=[]
    G.append(400*X[0]*(X[0]**2-X[1])+2*(X[0]-1))
    G.append(-200*(X[0]**2-X[1])+20.2*(X[1]-1)+19.8*(X[3]-1))
    G.append(2*(X[2]-1)+360*(X[2]**2-X[3])*X[2])
    G.append(-180*(X[2]**2-X[3])+20.2*(X[3]-1)+19.8*(X[1]-1))
    return np.array(G)
def Wood_Hessian(X):
    D1=[2+400*(X[0]**2-1)+800*X[0]**2,-400*X[0],0,0]
    D2=[-400*X[0],220.2,0,19.8]
    D3=[0,0,2+720*(X[2]**2)+360*(X[2]-X[3]),-360*X[2]]
    D4=[0,19.8,-360*X[2],200.2]
    return np.array([D1,D2,D3,D4])

Definimos las funciones para calcular el gradiente y Hessiano de a función de Branin.


In [113]:
a=1
b=5.1/(4*np.pi**2)
c=5/np.pi
r=6
s=10
t=1/(8*np.pi)
def Branin(X):
    return (a*(X[1]-b*X[0]**2+c*X[0]-r)**2+s*(1-t)*np.cos(X[0])+s)
def Branin_grad(X):
    return np.array([2*a*(X[1]-b*X[0]**2+c*X[0]-r)*(-2*X[0]+c)-s*(1-t)*np.sin(X[0]),2*a*(X[1]-b*X[0]**2+c*X[0]-r)])
def Branin_Hessian(X):
    R1=np.array([2*a*(X[1]-b*X[0]**2+c*X[0]-r)*(-2*b)+(-2*b*X[0]+c)*(-4*a*b*X[0]+2*a*c)-s*(1-t)*np.cos(X[0]),2*a*(-2*b*X[0]+c)])
    R2=np.array([2*a*(-2*b*X[0]+c),2*a])
    return np.array([R1,R2])

A continuncación implementamos el método de región de confianza usando la modificación de Dog-leg.

In [387]:
def get_Newton(grad,B):
    return np.linalg.solve(B,-grad)
def get_Cauchy(grad,B):
    return (grad*(-(np.dot(grad,grad))/(np.dot(grad,np.dot(B,grad)))))
def get_Newton_intersection(a,b,c):#Formula para hallar la interseccion
    return ((-b+np.sqrt(b**2-4*a*c))/(2*a))
def get_Cauchy_boundary(grad,H,radio):
    FC=np.dot(grad,np.dot(H,grad))
    if FC<=0:
        return -(radio/np.linalg.norm(grad))*grad#En la frontera
    else:
        tao=min(1,np.linalg.norm(grad)**3/(radio*FC))
        return -tao*(radio/np.linalg.norm(grad))*grad#Dentro de la region
def update_radio(radio_k,radio_global,rho,p_k):
    if rho<0.25:
        return 0.25*radio_k
    else:
        if rho>0.75:
            return np.min([2*radio_k,radio_global]) 
        else:
             return radio_k
    

In [418]:
def Dog_leg(x_0,f,grad_f,Hessian_f,radio_global,etha_1=0.25,tol_g=10**(-4),max_iter=10**3):
    x_k=x_0
    radio_k=radio_global
    for k in range(max_iter):
        grad_k=grad_f(x_k)
        f_k=f(x_k)
        H_k=Hessian_f(x_k)
        if(np.linalg.norm(grad_k)<tol_g):
            break
        #Conseguimos $p_k$
        p_k_B=get_Newton(grad_k,H_k)#Newton sin restricciones
        if (np.linalg.norm(p_k_B)>=radio_k):#Fuera de la region
            p_k_U=get_Cauchy(grad_k,H_k)
            if(np.linalg.norm(p_k_U)>=radio_k): #Fuera de la region
                #Buscamos que se satisfaga la restriccion de la region
                p_k= get_Cauchy_boundary(grad_k,H_k,radio_k)
            else:# p_k_U Dentro de la region 
                #Buscamos interseccion de Newton
                lamb=get_Newton_intersection(a=np.linalg.norm(p_k_B-p_k_U)**2,b=2*np.dot(p_k_U,p_k_B-p_k_U),c=np.linalg.norm(p_k_U)**2-radio_k**2)
                p_k=p_k_U+lamb*(p_k_B-p_k_U)
        else:
            p_k=p_k_B
        #Calcular medida de ajuste
        rho=(f_k-f(x_k+p_k))/(-(np.dot(grad_k,p_k)+0.5*np.dot(p_k,np.dot(H_k,p_k))))
        if(rho>etha_1):
            #Actualizamos x_k
            x_k_next=x_k+p_k
            x_k=x_k_next
        radio_k=update_radio(radio_k,radio_global,rho,p_k)
    return [x_k,k]

Implementamos el algoritmo de Newton-Cauchy alterno

In [419]:
def Cauchy_Newton(x_0,f,grad_f,Hessian_f,radio_global,etha_1=0.25,tol_g=10**(-4),max_iter=10**3):
    x_k=x_0
    radio_k=radio_global
    progr=[]
    for k in range(max_iter):
        grad_k=grad_f(x_k)
        f_k=f(x_k)
        H_k=Hessian_f(x_k)
        if(np.linalg.norm(grad_k)<tol_g):
            break
        #Conseguimos $p_k$
        p_k_B=get_Newton(grad_k,H_k)#Newton sin restricciones
        if (np.linalg.norm(p_k_B)>=radio_k):#Fuera de la region
            p_k=get_Cauchy_boundary(grad_k,H_k,radio_k)
        else:
            p_k=p_k_B
        #Calcular medida de ajuste
        rho=(f_k-f(x_k+p_k))/(-(np.dot(grad_k,p_k)+0.5*np.dot(p_k,np.dot(H_k,p_k))))
        if(rho>etha_1):
            #Actualizamos x_k
            x_k_next=x_k+p_k
            x_k=x_k_next
        radio_k=update_radio(radio_k,radio_global,rho,p_k)
        progr.append(f(x_k))
    return [x_k,k]

Implementamos el algoritmo de Newton Modificado. Utilizaremos un múltiplo de la identidad usando una aproximación del menor eigenvalor, aunque pueda ser computacionalmente costoso.

In [414]:
def min_eigen(A,tol,max_iter,pr=True,v_0=[]):#Codigo reciclado
    n=len(A)
    if(len(v_0)==0):#Es opcional
        v_0=(np.zeros(n)+1/np.sqrt(n))#Normalizado
    v_1=np.linalg.solve(A,v_0)#Se resuelve el sistema
    lambda_0=np.dot(v_0,v_1)
    lambda_1=np.sqrt(np.dot(v_1,v_1))#se toma raíz para poder compararlos
    count=0
    while((abs(lambda_0-lambda_1)>tol)&(count<max_iter)):
        count+=1
        v_0=v_1/np.linalg.norm(v_1)#Se normaliza
        v_1=np.linalg.solve(A,v_0)#iteracion
        lambda_0=lambda_1
        lambda_1=np.dot(v_0,v_1)
    if(count>=max_iter):
        print("Maxima iteracion alcanzada")
    else:
        if(pr==True):
            print("numero de iteraciones: "+str(count))
    return([1/lambda_1,v_1])
def get_alpha(f,grad_f,d,x_k,alpha_0=1,c_1=10**(-4),c_2=0.9,max_iter=10**3):#Bisection with Wolfe conditions
    alpha=0
    grad_f_k=grad_f(x_k)
    f_k=f(x_k)
    beta=np.inf
    alpha_i=alpha_0
    for i in range(max_iter):
        if(f(x_k+alpha_i*d)>f_k+c_1*alpha_i*np.dot(grad_f_k,d)):
            beta=alpha_i
            alpha_i=0.5*(alpha+beta)
        else:
            if(np.dot(grad_f(x_k+alpha_i*d),d)<c_2*np.dot(grad_f_k,d)):
                alpha=alpha_i
                if(beta==np.inf):
                    alpha_i=2*alpha
                else:
                    0.5*(alpha+beta)
            else:
                break
    return alpha_i

In [420]:
def Newton_mod(x_0,f,grad_f,Hessian_f,tol_g=10**(-4),max_iter=10**3):
    x_k=x_0
    alpha_k=10
    for j in range(max_iter):
        f_k=f(x_k)
        grad_k=grad_f(x_k)
        H_k=Hessian_f(x_k)
        if(np.linalg.norm(grad_k)<tol_g):
            break
        lamb_0=min_eigen(H_k,tol=10**(-6),max_iter=10**3,pr=False)[0]#Menor eigenvalor
        if(lamb_0<0):
            H_k=H_k-np.identity(len(x_0))*(lamb_0)
        #Calculamos la direccion
        d=np.linalg.solve(H_k,-grad_k)
        #Compute alpha_k
        alpha_k=get_alpha(f=f,grad_f=grad_f,x_k=x_k,d=d,alpha_0=1)
        x_k=x_k+alpha_k*d
    return [x_k,j]

In [421]:
print("Ejemplo: función de Wood\n")
n=4
np.random.seed(0)
x_0=np.random.uniform(-2,2,size=n)
print("Usando Cauchy-Newton:","f(x_k)=",Wood(Cauchy_Newton(x_0=x_0,f=Wood,grad_f=Wood_grad,Hessian_f=Wood_Hessian,radio_global=0.2,etha_1=0.25,tol_g=10**(-4),max_iter=10**3)[0]))
print("Usando Dog-leg:","f(x_k)=",Wood(Dog_leg(x_0=x_0,f=Wood,grad_f=Wood_grad,Hessian_f=Wood_Hessian,radio_global=0.05,etha_1=0.25,tol_g=10**(-4),max_iter=10**3)[0]))
print("Usando Newton modificado:","f(x_k)=",Wood(Newton_mod(x_0=x_0,f=Wood,grad_f=Wood_grad,Hessian_f=Wood_Hessian,tol_g=10**(-4),max_iter=10**3)[0]))

Ejemplo: función de Wood

Usando Cauchy-Newton: f(x_k)= 3.542045387404165e-12
Usando Dog-leg: f(x_k)= 3.555100497710087e-13
Usando Newton modificado: f(x_k)= 2.815556199461769


In [422]:
import numpy as np
from numba import njit
import time
from datetime import timedelta

In [437]:
print("Promedios de tiempo: función de Rosembrok\n")
n=100
T=3
tiempo_1=0
iter_1=0
tiempo_2=0
iter_2=0
tiempo_3=0
iter_3=0
for i in range(T):
        np.random.seed(0)
        x_0=np.random.uniform(-2,2,size=n)
        start = time.monotonic()
        C1 =Cauchy_Newton(x_0=x_0,f=Rosembrok,grad_f=Rosembrok_grad,Hessian_f=Rosembrok_Hessian,radio_global=0.05,etha_1=0.25,tol_g=10**(-4),max_iter=10**4)[1]
        end = time.monotonic()
        tiempo_1+=(timedelta(seconds=end - start).total_seconds())/T
        iter_1+=C1/T
        start = time.monotonic()
        C2 =Dog_leg(x_0=x_0,f=Rosembrok,grad_f=Rosembrok_grad,Hessian_f=Rosembrok_Hessian,radio_global=0.05,etha_1=0.25,tol_g=10**(-4),max_iter=10**4)[1]
        end = time.monotonic()
        tiempo_2+=(timedelta(seconds=end - start).total_seconds())/T
        iter_2+=C2/T
        C3 =Newton_mod(x_0=x_0,f=Rosembrok,grad_f=Rosembrok_grad,Hessian_f=Rosembrok_Hessian,tol_g=10**(-4),max_iter=10**4)[1]
        end = time.monotonic()
        tiempo_3+=(timedelta(seconds=end - start).total_seconds())/T
        iter_3+=C3/T

print("Usando Cauchy-Newton:", "Tiempo:",tiempo_1,'\n',"        ","Iteraciones:",iter_1)
print("Usando Dog-leg:", "Tiempo:",tiempo_2,'\n',"        ","Iteraciones:",iter_2)
print("Usando Newton modificado:", "Tiempo:",tiempo_3,'\n',"        ","Iteraciones:",iter_3)
print('-----------------------')


Promedios de tiempo: función de Rosembrok

Usando Cauchy-Newton: Tiempo: 5.714868333333333 
          Iteraciones: 9999.0
Usando Dog-leg: Tiempo: 0.7388269999999999 
          Iteraciones: 1114.0
Usando Newton modificado: Tiempo: 59.84583333333333 
          Iteraciones: 9999.0
-----------------------


In [438]:
print("Promedios de tiempo: función de Wood\n")
n=4
T=30
tiempo_1=0
iter_1=0
tiempo_2=0
iter_2=0
tiempo_3=0
iter_3=0
for i in range(T):
        np.random.seed(0)
        x_0=np.random.uniform(-2,2,size=n)
        start = time.monotonic()
        C2 =Cauchy_Newton(x_0=x_0,f=Wood,grad_f=Wood_grad,Hessian_f=Wood_Hessian,radio_global=0.05,etha_1=0.25,tol_g=10**(-4),max_iter=10**4)[1]
        end = time.monotonic()
        tiempo_1+=(timedelta(seconds=end - start).total_seconds())/T
        iter_1+=C1/T
        start = time.monotonic()
        C2 =Dog_leg(x_0=x_0,f=Wood,grad_f=Wood_grad,Hessian_f=Wood_Hessian,radio_global=0.05,etha_1=0.25,tol_g=10**(-4),max_iter=10**4)[1]
        iter_2+=C2/T
        end = time.monotonic()
        tiempo_2+=(timedelta(seconds=end - start).total_seconds())/T
        C2 =Newton_mod(x_0=x_0,f=Wood,grad_f=Wood_grad,Hessian_f=Wood_Hessian,tol_g=10**(-4),max_iter=10**4)[1]
        iter_3+=C3/T
        end = time.monotonic()
        tiempo_3+=(timedelta(seconds=end - start).total_seconds())/T
        
print("Usando Cauchy-Newton:", "Tiempo:",tiempo_1,'\n',"        ","Iteraciones:",iter_1)
print("Usando Dog-leg:", "Tiempo:",tiempo_2,'\n',"        ","Iteraciones:",iter_2)
print("Usando Newton modificado:", "Tiempo:",tiempo_3,'\n',"        ","Iteraciones:",iter_3)
print('-----------------------')


Promedios de tiempo: función de Wood

Usando Cauchy-Newton: Tiempo: 0.005123566666666667 
          Iteraciones: 9998.999999999998
Usando Dog-leg: Tiempo: 0.004549666666666665 
          Iteraciones: 47.00000000000003
Usando Newton modificado: Tiempo: 5.977737800000001 
          Iteraciones: 9998.999999999998
-----------------------


In [444]:
print("Promedios de tiempo: función de Branin\n")
n=2
T=30
tiempo_1=0
iter_1=0
tiempo_2=0
iter_2=0
tiempo_3=0
iter_3=0
for i in range(T):
        np.random.seed(0)
        x_0=np.random.uniform(-2,2,size=n)
        start = time.monotonic()
        C1 =Cauchy_Newton(x_0=x_0,f=Branin,grad_f=Branin_grad,Hessian_f=Branin_Hessian,radio_global=0.1,etha_1=0.25,tol_g=10**(-4),max_iter=10**4)[1]
        iter_1+=C1/T
        end = time.monotonic()
        tiempo_1+=(timedelta(seconds=end - start).total_seconds())/T
        start = time.monotonic()
        C2 =Dog_leg(x_0=x_0,f=Branin,grad_f=Branin_grad,Hessian_f=Branin_Hessian,radio_global=0.05,etha_1=0.25,tol_g=10**(-4),max_iter=10**4)[1]
        iter_2+=C2/T
        end = time.monotonic()
        tiempo_2+=(timedelta(seconds=end - start).total_seconds())/T
        C2 =Newton_mod(x_0=x_0,f=Branin,grad_f=Branin_grad,Hessian_f=Branin_Hessian,tol_g=10**(-4),max_iter=10**3)[1]
        iter_3+=C3/T
        end = time.monotonic()
        tiempo_3+=(timedelta(seconds=end - start).total_seconds())/T
        
print("Usando Cauchy-Newton:", "Tiempo:",tiempo_1,'\n',"        ","Iteraciones:",iter_1)
print("Usando Dog-leg:", "Tiempo:",tiempo_2,'\n',"        ","Iteraciones:",iter_2)
print("Usando Newton modificado:", "Tiempo:",tiempo_3,'\n',"        ","Iteraciones:",iter_3)
print('-----------------------')


Promedios de tiempo: función de Branin



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in double_scalars
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars


Usando Cauchy-Newton: Tiempo: 0.8709520999999998 
          Iteraciones: 9998.999999999998
Usando Dog-leg: Tiempo: 0.9555218 
          Iteraciones: 9998.999999999998
Usando Newton modificado: Tiempo: 1.739676933333333 
          Iteraciones: 9998.999999999998
-----------------------
